In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [4]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import misc_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
from pathlib import Path
import os
import requests
import ipywidgets as widgets

### Define Parameters for Data Loading

In [5]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [6]:
fc = FileChooser()
fc.filter_pattern = '*.czi'
display(fc)

FileChooser(path='E:\Github\czitools\demo\notebooks', filename='', title='', show_hidden=False, select_desc='S…

In [7]:
# Print the selected path, filename, or both
print(fc.selected_path)
print(fc.selected_filename)
print(fc.selected)

E:\Github\czitools\data
CellDivision_T10_Z15_CH2_DCV_small.czi
E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small.czi


In [8]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(fc.selected)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

2024-02-11 13:00:54,273 -  INFO - Reading Dimensions from CZI image data.


SizeS:  None
SizeT:  10
SizeZ:  15
SizeC:  2
SizeY:  256
SizeX:  256


In [9]:
# and get more info
czi_scaling = czimd.CziScaling(fc.selected)
czi_channels = czimd.CziChannelInfo(fc.selected)
czi_bbox = czimd.CziBoundingBox(fc.selected)
czi_objectives = czimd.CziObjectives(fc.selected)
czi_detectors = czimd.CziDetector(fc.selected)
czi_microscope = czimd.CziMicroscope(fc.selected)
czi_sample = czimd.CziSampleInfo(fc.selected)

2024-02-11 13:00:57,215 -  INFO - Reading Scaling from CZI image data.
2024-02-11 13:00:57,455 -  INFO - Reading Channel Information from CZI image data.
2024-02-11 13:00:57,707 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-11 13:00:57,708 -  INFO - Reading Objective Information from CZI image data.
2024-02-11 13:00:57,935 -  INFO - Reading Detector Information from CZI image data.
2024-02-11 13:00:58,148 -  INFO - Reading Microscope Information from CZI image data.
2024-02-11 13:00:58,356 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-11 13:00:58,561 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 13:00:58,561 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.
2024-02-11 13:00:58,776 -  INFO - Reading Dimensions from CZI image data.


In [10]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(fc.selected)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc_tools.md2dataframe(mdict)

# create a ipywdiget to show the dataframe with the metadata
wd1 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd1:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd1]))

2024-02-11 13:01:01,739 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 13:01:01,742 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-11 13:01:01,743 -  INFO - Reading Channel Information from CZI image data.
2024-02-11 13:01:01,744 -  INFO - Reading Scaling from CZI image data.
2024-02-11 13:01:01,745 -  INFO - Reading Objective Information from CZI image data.
2024-02-11 13:01:01,745 -  INFO - Reading Detector Information from CZI image data.
2024-02-11 13:01:01,746 -  INFO - Reading Microscope Information from CZI image data.
2024-02-11 13:01:01,748 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-11 13:01:01,749 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 13:01:01,751 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.
2024-02-11 13:01:01,982 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 13:01:01,991 -  INFO - Reading additional Metedata from CZI image data.
2

In [11]:
# write XML to disk
xmlfile = czimd.writexml(fc.selected)
print("XML File written to:", xmlfile)

XML File written to: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_CZI_MetaData.xml


In [12]:
# get the planetable for the CZI file
pt, csvfile = misc_tools.get_planetable(fc.selected,
                                  norm_time=True,
                                  savetable=True,
                                  separator=',',
                                  index=True)

# create a ipywdiget to show the dataframe with the metadata
wd2 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd2:
    display(HTML(pt.to_html()))
display(widgets.VBox(children=[wd2]))

print("PlaneTable CSV File written to:", csvfile)

2024-02-11 13:01:14,270 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 13:01:14,835 -  INFO - PlaneTable saved as CSV file: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_planetable.csv


PlaneTable CSV File written to: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_planetable.csv
